# Analyzing Assessed Value of the Downtown Mall

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import requests
import folium
from folium import plugins
import geojson

In [2]:
#Import .xls retrieved from the GIS Viewer into pandas

#Path to the .xls retrieve from the GIS Viewer
f = r'./data/mall_parcels_no_ends.xls'
#Create a dataframe that reads the .xls file
df = pd.read_excel(f)
#Identify all rows in df where MULTIPIN column is not equal to 1
not_multipin = df['MULTIPIN'] != 1
#Create a new dataframe that only contains the rows identified in not_multipin
df = df[not_multipin]

In [3]:
## Create Query URL's for open portal data based on .xls data

formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

# Url for Parcel Area Boundary geojson data
parcel_url = f'https://opendata.arcgis.com/datasets/320d465ddf0d498796da6491e21f6dde_43.geojson?where=GPIN%20in%20({formatted_gpins})'

formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins = formatted_pins
formatted_pins = ','.join(formatted_pins)

assessment_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"
assessment_request = requests.get(assessment_url)
assessment_json = assessment_request.json()

In [4]:
# Convert geojson parcel boundary data to dataframe
# Turn geojson data into dataframe
geojson_df = pd.read_json(parcel_url)
geojson_features_df = geojson_df['features']
geojson_features_df_generator = (x for x in geojson_features_df)
ft_list = []
for x in geojson_features_df_generator:
    ft_list.append([x['properties']['GPIN'],x['geometry']['coordinates']])
geojson_df = pd.DataFrame(ft_list, columns = ['GPIN','coordinates'])
ft_list = []
geojson_df['GPIN_str'] = pd.DataFrame(geojson_df['GPIN'], dtype = 'str')
geojson_df = geojson_df.drop('GPIN', axis = 1)
geojson_df['GPIN'] = pd.DataFrame(geojson_df['GPIN_str'], dtype = 'str')
geojson_df = geojson_df.drop('GPIN_str', axis = 1)
#------------
# Create data series based on features and combine data frames into a single df
assessment_df = pd.DataFrame(assessment_json['features'])
# Create a single data frame based on combined series data
assessments = pd.DataFrame([x for x in assessment_df['attributes']], dtype = 'object')
assessments = assessments.astype({'TaxYear': 'int64', 'ImprovementValue': 'int64','LandValue': 'int64','TotalValue': 'int64', 'ParcelNumber': 'str'})
assessments = assessments.drop(['ImprovementValue', 'LandValue'], axis=1)
#----------------
#Create data frame from df that holds PIN and GPIN.
df_key = pd.DataFrame(df[['PIN','GPIN']], dtype = 'str')
#----------------
# Merge df_key with assessments
d = pd.merge(assessments, df_key, how='inner', left_on=['ParcelNumber'], right_on=['PIN'])
#----------------
## Merge geojson_df with d
geojson_assessment_df = d.merge(geojson_df, on = 'GPIN')
geojson_assessment_df = geojson_assessment_df.drop(['ParcelNumber'], axis=1)
geojson_assessment_df.head()

,TaxYear,TotalValue,PIN,GPIN,coordinates
0,2019,5963500,280001000,6861,"[[[-78.48280641299056, 38.03075462954861], [-7..."
1,2018,5721100,280001000,6861,"[[[-78.48280641299056, 38.03075462954861], [-7..."
2,2017,5795845,280001000,6861,"[[[-78.48280641299056, 38.03075462954861], [-7..."
3,2016,4000500,280001000,6861,"[[[-78.48280641299056, 38.03075462954861], [-7..."
4,2015,3795600,280001000,6861,"[[[-78.48280641299056, 38.03075462954861], [-7..."


In [5]:
#Store year_min, max, and a generator of years into variables
year_min = int(geojson_assessment_df['TaxYear'].min())
year_max = int(geojson_assessment_df['TaxYear'].max())
years = (x for x in range(year_min, year_max+1))

In [9]:
# Use this to create a new geojson dataset for map application

#https://gis.stackexchange.com/questions/220997/pandas-to-geojson-multiples-points-features-with-python
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Polygon((X["coordinates"]),
                            properties=dict(name=X["GPIN"],
                            description=X["TotalValue"]))))
    df.apply(insert_features, axis=1)
    with open('map1.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

data2geojson(geojson_assessment_df)
'''
styledict = {
    '0': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        },
    '1': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        }
}
'''

styledict = {
    '0': {
        '2018': {'color': 'ffffff', 'opacity': 1}
        },
    '1': {
        '2019': {'color': '000000', 'opacity': 1}
        }
}

m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')
#folium.GeoJson(r'./map1.geojson',name='geojson').add_to(m)
folium.plugins.TimeSliderChoropleth(r'./map1.geojson',styledict = styledict, name='Time Slider Demo', overlay=True, control=True, show=True)
m

In [7]:
def all_values_by_year(year):
    all_values_df = geojson_assessment_df[geojson_assessment_df['TaxYear'] == year]
    all_values_df = all_values_df.astype({'GPIN': 'int', 'TotalValue': 'int'})
    return all_values_df

In [8]:
## Map assessed value by year
def choropleth_assessed_val_by_year(year):
    m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')
    folium.Choropleth(
        geo_data = parcel_url,
        #geo_data = r'./map1.geojson',
        data=all_values_by_year(year)[['GPIN', 'TotalValue']],
        columns=['GPIN', 'TotalValue'], 
        key_on='feature.properties.GPIN', 
        bins=[0, 250000, 500000, 1000000, 2000000, 5000000, 10000000, 16000000],
        fill_color= 'YlOrRd',
        nan_fill_color='black', 
        fill_opacity=0.9, 
        nan_fill_opacity=1, 
        line_color='black', 
        line_weight=1, 
        line_opacity=1, 
        name='Assessed Value',
        legend_name= f'Total Assessed Value {year}'
    ).add_to(m)
    return(m)

choropleth_assessed_val_by_year(2019)